# Welcome to the Notebooks Container Runtime!

In this notebook, we will go through the basics of using Notebooks Container Runtime. We will install packages, load data, train a model, and look at logs. 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Add a query tag to the session. This helps with debugging and performance monitoring.
session.query_tag = {"origin":"sf_sit", "name":"notebook_pack_container_runtime", "version":{"major":1, "minor":0}}

# Set session context 
session.use_role(f"{{ env.DATAOPS_CATALOG_SOLUTION_PREFIX }}_DATA_SCIENTIST") 

# Print the current role, warehouse, and database/schema
print(f"role: {session.get_current_role()} | WH: {session.get_current_warehouse()} | DB.SCHEMA: {session.get_fully_qualified_current_schema()}")

The Container Runtime for Snowflake Notebooks includes pre-installed common packages including SnowparkML and other OSS packages.

In [ ]:
!pip freeze

Notebooks Container Runtime, along with External Access Integrations give us the flexibility to `pip install` packages from anywhere, including popular package repositories such as pypi. You can install whatever packages you need by running `!pip install <package_name>` directly in the Notebook.

We have configured this notebook to allow pypi urls with an External Access Integration. 

In [ ]:
!pip install seaborn

Just like Notebooks on the Warehouse Runtime, we can intermingle both SQL and Python cells:

In [ ]:
show tables;

Let's visualize some of our data using the `seaborn` package that we installed above:

In [ ]:
diamonds_df = session.table("DIAMONDS")
diamonds_df.show()

In [ ]:
df = diamonds_df.to_pandas()

import seaborn as sns

# Create a visualization
sns.histplot(
    data=df,
    x="PRICE"
)

Now, let's train a basic `XGBRegressor` machine learning model. The ML Container Runtime for Snowflake Notebooks includes pre-installed common packages for doing machine learning tasks, including SnowparkML and other OSS packages.

In [ ]:
import time
from snowflake.ml.modeling.xgboost import XGBRegressor

CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]
NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "X", "Y", "Z"]
LABEL_COLUMNS = ['PRICE']
diamonds_df = session.table("diamonds")

model = XGBRegressor(max_depth=400, input_cols=NUMERICAL_COLUMNS, label_cols=LABEL_COLUMNS)

t0 = time.time()
model.fit(diamonds_df)

t1 = time.time()

print(f"Fit in {t1-t0} seconds.")